In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import librosa
import librosa.display
from scipy.signal import lfilter
from scipy.signal.windows import hamming

from numpy.linalg import lstsq

%matplotlib inline

from google.colab import drive
drive.mount('/content/drive')

zip_path="/content/drive/MyDrive/L2_DATASET.zip"

import zipfile,os,glob
import numpy as np
import librosa, librosa.display
import matplotlib.pyplot as plt
from IPython.display import Audio, display

extract_path="/content/voices_dataset"
os.makedirs(extract_path,exist_ok=True)
with zipfile.ZipFile(zip_path,'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("Dataset unzipped to:",extract_path)
print("Files:",os.listdir(extract_path))

print("Extracted folder content:")
for r,ds,fs in os.walk(extract_path):
    print(r,"→",fs)

#  dataset paths ---
extract_path="/content/voices_dataset"
audio_dir=os.path.join(extract_path,"L2_DATASET")
all_audio=sorted(glob.glob(os.path.join(audio_dir,"*.wav")))
print(f"Found {len(all_audio)} WAV files")
assert all_audio, "No WAV files found. Check /content/voices_dataset/L2_DATASET"


# Helper: LPC-based formant estimation ----
def lpc_formants(sig, sr, order=10):
    sig = sig * hamming(len(sig))
    # Build prediction matrix A
    A = np.zeros((len(sig)-order, order))
    for i in range(order):
        A[:, i] = sig[order-i-1:len(sig)-i-1]
    y_vec = sig[order:]
    a, _, _, _ = lstsq(A, y_vec, rcond=None)
    roots = np.roots(np.concatenate(([1], -a)))
    roots = [r for r in roots if np.imag(r) >= 0]  # keep upper half-plane
    angles = np.arctan2(np.imag(roots), np.real(roots))
    formants = sorted(angles * (sr / (2*np.pi)))
    return formants[:3]  # return first three formants

# Analyze one sample file ----
first_file=all_audio[0]
y, sr=librosa.load(first_file, sr=None, mono=True)
display(Audio(y, rate=sr))

# pick ~200 ms segment from middle
segment=y[int(0.5*sr):int(0.7*sr)]
if len(segment)==0:
    print("Segment empty. Adjust start/end times.")
else:
    formants=lpc_formants(segment, sr)
    print(f"{os.path.basename(first_file)} → Estimated Formants (Hz): {[f'{f:.1f}' for f in formants]}")
    print("F1 and F2 are key for vowel identification.")

# Loop through all files ----
results=[]
for fp in all_audio:
    y, sr=librosa.load(fp, sr=None, mono=True)
    segment=y[int(0.5*sr):int(0.7*sr)]
    if len(segment)==0:
        continue
    formants=lpc_formants(segment, sr)
    results.append((os.path.basename(fp), formants[:3]))
    print(f"{os.path.basename(fp)} → F1={formants[0]:.1f} Hz, F2={formants[1]:.1f} Hz, F3={formants[2]:.1f} Hz")

# save results to text file ----
out_path=os.path.join(extract_path,"formant_results.txt")
with open(out_path,"w") as f:
    for name, fvals in results:
        f.write(f"{name}: {', '.join([f'{fv:.1f}' for fv in fvals])}\n")

print(f"\n Saved formant results to: {out_path}")
